In [ ]:
import os
import glob
import time
import numpy
import pickle
import tensorflow as tf

from music21 import converter, instrument, note, chord, stream
from keras.utils import np_utils

In [ ]:
# Configuration
prediction_length = 250      # Number of Notes to Predict
sequence_length = 100        # LSTM Sequence Length
tempo = 0.5                  # Pause between Notes
model_name = 'weights-199-0.4371' # Model Name

In [ ]:
notes = []

In [ ]:
with open('dataset/notes/parsed.data', 'rb') as filepath:
    notes = pickle.load(filepath)

pitchnames = sorted(set(item for item in notes))
n_vocab = len(set(notes))

In [ ]:
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    output.append(note_to_int[sequence_out])

n_patterns = len(network_input)


normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

normalized_input = normalized_input / float(n_vocab)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(
        512, 
        input_shape=(normalized_input.shape[1], normalized_input.shape[2]), 
        recurrent_dropout=0.3, 
        return_sequences=True
    ),
    tf.keras.layers.LSTM(512, return_sequences=True, recurrent_dropout=0.3),
    tf.keras.layers.LSTM(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Activation(tf.keras.activations.relu),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(n_vocab),
    tf.keras.layers.Activation(tf.keras.activations.softmax)
])

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.load_weights('models/' + model_name + '.hdf5')

In [ ]:
start = numpy.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

for note_index in range(prediction_length):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += tempo

In [ ]:
model_id = model_name.split('-')[1]
os.makedirs('output/' + model_id, exist_ok=True)

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output/' + model_id + '/output-' + str(int(time.time())) + '.mid')